In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import *   # 引入sqrt(), pi, exp等
import cmath  # 要处理复数情况，用到cmath.exp(), 比np.快
import time

In [2]:
def find_vector_without_excess_phase(vector_0):  # 使波函数矢量第一分量为实数，同时归一化
    angle = cmath.phase(vector_0[0])
    vector_1 = vector_0*cmath.exp(-1j*angle)
    vector_1 = vector_1/cmath.sqrt(np.dot(vector_1.transpose().conj(),vector_1))
    return vector_1

In [3]:
def get_data_kx_ky_Ez(route):                # 由数据文件得到kx,ky,Ez阵列
    with open(route,'r') as file:
        data_row = file.readlines()
    data1 = data_row[8]   # 文字部分
    data2 = data_row[9:]   # 数据部分
    
    #提取kx,ky
    data = data1.split('@')[1:]
    data = [item.split('=')[2:] for item in data]
    k1 = [float(item[0].split(',')[0]) for item in data]
    k2 = [float(item[1].split()[0]) for item in data]
    for i in range(1,len(k1)):
        if k1[i] != k1[0]:
            break
    kx = np.array(k1[::i])
    ky = np.array(k2[:i])
    kx_num = kx.shape[0] 
    ky_num = ky.shape[0]
    
    # ， 提取Ez
    data = [item.split()[2:] for item in data2]
    vector_len = len(data)
    Ez = np.zeros((kx_num,ky_num,vector_len),dtype=complex)
    
    for j in range(len(data[0])):   # len(data[0]) = kx_num * ky_num
        for i in range(vector_len):        
            try:
                Ez[j//ky_num][j%ky_num][i] = complex(data[i][j])
            except:
                Ez[j//ky_num][j%ky_num][i] = complex(data[i][j][:-1]+'j')  # Python中复数用j而不是i，直接转换会报错
                       
    for i in range(kx_num):
        for j in range(ky_num):
            v = Ez[i][j]
            v = find_vector_without_excess_phase(v)
            Ez[i][j] = v
       
    return kx, ky, Ez



In [4]:
def Polarization():        # 定义公式推导近似后方法,可以在整个布里渊区积分，也可以对固定的某个ky积kx，不影响结果
    z = 1
    kx, ky, Ez = get_data_kx_ky_Ez(route1)
    for i in range(kx.shape[0]-1):
        for j in range(ky.shape[0]):
            A_x = np.dot(Ez[i][j].transpose().conj(),Ez[i+1][j])    
            z = np.dot(z,A_x)    
    z = np.log(z)/2/pi/1j
    return z     

In [5]:
def Polarization_row():        # 定义法，需要两个扫频靠近的电场文件
    z = 0
    kx, ky, Ez = get_data_kx_ky_Ez(route1)
    kx_delta,ky_delta,Ez_delta = get_data_kx_ky_Ez(route2)
    delta = 2*pi*(kx_delta[0]-kx[0])
    for i in range(kx.shape[0]):
        for j in range(ky.shape[0]):
            A_x = np.dot(Ez[i][j].transpose().conj(),(Ez_delta[i][j]-Ez[i][j])/delta)   # Berry conection 的x分量
            #if i%5 == 0 and j%5 == 0:
                #print(A_x)
            z = z + A_x/kx.shape[0]/ky.shape[0]/1j
    return z

In [6]:
# 经尝试，方格子元胞k空间取20*20个点左右，Ez数据山哥导出，4*2个点（2*2也行，影响不大）
def main():
    z = Polarization()
    print('Zak phase = {:.2f} pi \nPolarization = {:.2f}'.format(z*2,z))
    
if __name__ == '__main__'
    route1 = 'd:/物理/拓扑/拓扑数计算python代码/数据/c4_expand_1.txt'        # 数字为能带标号，路径要修改，放在同一文件夹下不写绝对路径即可
    #route1 = 'd:/物理/拓扑/拓扑数计算python代码/数据/扩张0.txt'    
    #route2 = 'd:/物理/拓扑/拓扑数计算python代码/数据/扩张0_delta.txt'         # 如果使用定义法，则需替换为这两条
    main()

Zak phase = -1.00+2.21j pi 
Polarization = -0.50+1.11j
